In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob
import csv

# Read Data

In [2]:
# Collect Survey csv files
folder = "../../../data/surveys/responses/"
response_list = glob.glob(F"{folder}*")
folder = "../../../data/surveys/surveys/"
survey_list = [F"{folder}{lst[59:]}" for lst in response_list]

In [3]:
# Surveys
df_surveys = pd.concat((pd.read_csv(f, sep='\t') for f in survey_list), ignore_index=True)
df_surveys.rename(columns={ df_surveys.columns[0]: "Index" }, inplace = True)

# Response
df_responses = pd.concat((pd.read_csv(f, header=None).iloc[: , 1:].T for f in response_list))
df_responses.rename(columns={ df_responses.columns[0]: "Sentence", 
                            df_responses.columns[1]: "Result" }, 
                            inplace = True)

## Testing

In [41]:
for survey, response in zip(survey_list, response_list):

    # Survey
    df_survey = pd.read_csv(survey, sep='\t') 

    # Response
    df_response = pd.read_csv(response, header=None).iloc[: , 1:].T
    df_response.columns = ['Sentence', 'Result']

    df_survey = pd.melt(df_survey, 
                        id_vars=["Species", "Main Trait", "SIM", "Dataset"],
                        value_vars=["1", "2", "3", "4", "5",],
                        value_name="Sentence")\
                    .dropna()\
                    .drop(columns=["variable"])
    
    df_merged = df_survey.merge(df_response, on='Sentence')
    try:
        df = pd.concat([df, df_merged], ignore_index=True, sort=False)
    except:
        df = df_merged

In [45]:
folder = "../../../data/surveys/"
df.to_csv(F"{folder}df_survey_result.csv")

## Before

In [4]:
# Melt
df_surveys = pd.melt(df_surveys, id_vars=["Species",
                                       "Main Trait",
                                       "SIM",
                                       "Dataset"],
                             value_vars=["1", "2", "3", "4", "5",],
                             value_name="Sentence"
                )\
                .dropna()\
                .drop(columns=["variable"])

In [5]:
# Merge the dataframes based on the 'Sentence' column
# df = pd.merge(df_surveys, df_responses, 
#               on='Sentence', 
#               how='inner'
#               )

df = df_surveys.sort_values(by=['Sentence'])
df["Result"] = df_responses.sort_values(by=['Sentence'])["Result"].values


In [12]:
df = df.drop_duplicates()

In [13]:
folder = "../../../data/surveys/"
df.to_csv(F"{folder}df_survey_result.csv")